In [1]:
import time

import numpy as np

CE = np.load('/workspace/data/EEG/data/epochs/A-epo.npy')
lab = np.load("/workspace/data/EEG/data/epochs/A-labels.npy")

In [2]:
#rCE = CE.reshape(CE.shape[0], CE.shape[1], CE.shape[3], CE.shape[2])
tCE = np.transpose(CE,(0,1,3,2))

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

def SepConv1D(Chans = 6, Samples = 206, Filters = 32):
  eeg_input    = Input(shape = (Samples, Chans))

  padded       = ZeroPadding1D(padding = 4)(eeg_input)
  block1       = SeparableConv1D(Filters, 16, strides = 8,
                                 padding = 'valid',
                                 data_format = 'channels_last',
                                 kernel_initializer = 'glorot_uniform',
                                 bias_initializer = 'zeros',
                                 use_bias = True)(padded)
  block1       = Activation('tanh')(block1)
  flatten      = Flatten(name = 'flatten')(block1)
  prediction   = Dense(1, activation = 'sigmoid')(flatten)

  return Model(inputs = eeg_input, outputs = prediction, name='SepConv1D')

In [4]:
# TODO: Add Class weights

In [30]:
import argparse
import sys
import numpy as np
#from SepConv1D import SepConv1D
from tensorflow.keras.callbacks import EarlyStopping
#from tensorflow import set_random_seed
from sklearn.model_selection import *
from utils import *
import tensorflow.keras.backend as K

def evaluate_subject_models(data, labels, modelpath, subject, n_filters = 32):
    """
    Trains and evaluates P300-CNNT for each subject in the P300 Speller database
    using repeated stratified K-fold cross validation.
    """
    n_sub = data.shape[0]
    n_trials = data.shape[1]
    n_samples = data.shape[2]
    n_channels = data.shape[3]

    inf_time = np.zeros(5 * 10)
    aucs = np.zeros(5 * 10)
    print("Training for subject {0}: ".format(subject))
    cv = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 10, random_state = 123)
    for k, (t, v) in enumerate(cv.split(data[subject], labels[subject])):
        X_train, y_train, X_test, y_test = data[subject, t, :, :], labels[subject, t], data[subject, v, :, :], labels[subject, v]
        X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.2, shuffle = True, random_state = 456)
        print('Partition {0}: X_train = {1}, X_valid = {2}, X_test = {3}'.format(k, X_train.shape, X_valid.shape, X_test.shape))

        # channel-wise feature standarization
        sc = EEGChannelScaler(n_channels = n_channels)
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)

        #tt = TrainTime()

        model = SepConv1D(Chans = n_channels, Samples = n_samples, Filters = n_filters)
        print(model.summary())
        model.compile(optimizer = 'adam', loss = 'binary_crossentropy')

        # Early stopping setting also follows EEGNet (Lawhern et al., 2018)
        es = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 50, restore_best_weights = True)

        start_train = time.time()
        history = model.fit(X_train,
                            y_train,
                            batch_size = 256,
                            epochs = 200,
                            validation_data = (X_valid, y_valid),
                            callbacks = [es])
        train_time = time.time()-start_train

        start_test = time.time()
        proba_test = model.predict(X_test)
        test_time =  time.time() - start_test

        test_size = X_test.shape[0]
        inf_time[k] = test_time/test_size

        aucs[k] = roc_auc_score(y_test, proba_test)
        print('S{0}, P{1} -- AUC: {2}'.format(subject, k, aucs[k]))
        K.clear_session()

    #train_size = X_train.shape[0]
    #valid_size = X_valid.shape[0]
    #test_size =  X_test.shape[0]

    #times = [[np.mean(tt.times), np.sum(tt.times), 10, train_size, valid_size, test_time, test_size, test_time / test_size]]
    ##df = pd.DataFrame(times, columns = ['Mean Epoch Time', 'Total Train Time', 'Epochs', 'Train Size', 'Valid Size', 'Test Time', 'Test Size', 'Test per example'])
    #df.to_csv(modelpath + 'SepConv1D_' + str(n_filters) + 'F_times.csv', encoding='utf-8')



    np.savetxt(modelpath + '/s' + str(subject) + '_auc.npy', aucs)
    np.savetxt(modelpath + '/inf_time.npy', inf_time)

    np.save(modelpath + '/s' + str(subject) + '_data.npy', X_test)
    np.save(modelpath + '/s' + str(subject) + '_labels.npy', y_test)
    model.save_weights(modelpath + '/s' + str(subject) + '_model.h5')

    return aucs, train_time, inf_time

In [19]:
import time
def evaluate_cross_subject_model(data, labels, modelpath, n_filters = 32):
    """
    Trains and evaluates SepConv1D for each subject in the P300 Speller database
    using random cross validation.
    """
    n_sub = data.shape[0]
    n_ex_sub = data.shape[1]
    n_samples = data.shape[2]
    n_channels = data.shape[3]

    aucs = np.zeros(n_sub)
    inf_time = np.zeros(n_sub)


    data = data.reshape((n_sub * n_ex_sub, n_samples, n_channels))
    labels = labels.reshape((n_sub * n_ex_sub))
    groups = np.array([i for i in range(n_sub) for j in range(n_ex_sub)])

    cv = LeaveOneGroupOut()
    for k, (t, v) in enumerate(cv.split(data, labels, groups)):
        X_train, y_train, X_test, y_test = data[t], labels[t], data[v], labels[v]
        rg = np.random.choice(t, 1)
        sv = groups[t] == groups[rg]
        st = np.logical_not(sv)
        X_train, y_train, X_valid, y_valid = data[t][st], labels[t][st], data[t][sv], labels[t][sv]
        print("Partition {0}: train = {1}, valid = {2}, test = {3}".format(k, X_train.shape, X_valid.shape, X_test.shape))
        print("Groups train = {0}, valid = {1}, test = {2}".format(np.unique(groups[t][st]),
                                                                   np.unique(groups[t][sv]),
                                                                   np.unique(groups[v])))

         # channel-wise feature standarization
        sc = EEGChannelScaler(n_channels = n_channels)
        X_train = sc.fit_transform(X_train)
        X_valid = sc.transform(X_valid)
        X_test = sc.transform(X_test)

        model = SepConv1D(Chans = n_channels, Samples = n_samples, Filters = n_filters)
        print(model.summary())
        model.compile(optimizer = 'adam', loss = 'binary_crossentropy')

        es = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 50, restore_best_weights = True)
        start_train = time.time()
        model.fit(X_train,
                  y_train,
                  batch_size = 256,
                  epochs = 200,
                  validation_data = (X_valid, y_valid),
                  callbacks = [es])
        train_time = time.time()-start_train

        start_test = time.time()
        proba_test = model.predict(X_test)
        test_time = time.time() - start_test

        test_size = X_test.shape[0]
        inf_time[k] = test_time/test_size


        aucs[k] = roc_auc_score(y_test, proba_test)
        print('P{0} -- AUC: {1}'.format(k, aucs[k]))
        model.save_weights(modelpath + '/s' + str(np.unique(groups[v])[0]) + '_model.h5')
        np.save(modelpath + '/s' + str(np.unique(groups[v])[0]) + '_data.npy', X_test)
        np.save(modelpath + '/s' + str(np.unique(groups[v])[0]) + '_labels.npy', y_test)
        K.clear_session()

    np.savetxt(modelpath + '/aucs.npy', aucs)
    np.savetxt(modelpath + '/inf_time.npy', inf_time)

    return aucs, train_time, inf_time

In [15]:
within_modelpath = "/workspace/data/EEG/models/sepconv1d/within/"
cross_modelpath = "/workspace/data/EEG/models/sepconv1d/cross/"

In [31]:
auc_Wsub, Wtrain_time, Winf_time = evaluate_subject_models(tCE, lab, within_modelpath, 0)

Training for subject 0: 
Partition 0: X_train = (2688, 257, 8), X_valid = (672, 257, 8), X_test = (840, 257, 8)
Model: "SepConv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 257, 8)]          0         
                                                                 
 zero_padding1d (ZeroPadding  (None, 265, 8)           0         
 1D)                                                             
                                                                 
 separable_conv1d (Separable  (None, 32, 32)           416       
 Conv1D)                                                         
                                                                 
 activation (Activation)     (None, 32, 32)            0         
                                                                 
 flatten (Flatten)           (None, 1024)              0         
           

In [32]:
np.mean(auc_Wsub)


0.8758136734693878

In [21]:
auc_Csub, train_time, inf_time = evaluate_cross_subject_model(tCE, lab, cross_modelpath, n_filters = 32)

Partition 0: train = (25200, 257, 8), valid = (4200, 257, 8), test = (4200, 257, 8)
Groups train = [1 2 3 4 5 6], valid = [7], test = [0]
Model: "SepConv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 257, 8)]          0         
                                                                 
 zero_padding1d_2 (ZeroPaddi  (None, 265, 8)           0         
 ng1D)                                                           
                                                                 
 separable_conv1d_2 (Separab  (None, 32, 32)           416       
 leConv1D)                                                       
                                                                 
 activation_2 (Activation)   (None, 32, 32)            0         
                                                                 
 flatten (Flatten)           (None, 1024)          

In [22]:
np.mean(auc_Csub)


0.7727336479591838

In [23]:
train_time

(21.876333475112915,
 array([7.35154038e-05, 7.90540377e-05, 6.90653778e-05, 8.09862500e-05,
        6.80619194e-05, 7.53264200e-05, 6.88177631e-05, 2.40089269e-04]))

In [24]:
train_time


21.876333475112915

In [29]:
np.mean(inf_time)*60

0.005661873306546892

In [26]:
inf_time.shape

(8,)

In [28]:
9.436455510911486e-05

9.436455510911486e-05

In [33]:
Wtrain_time


4.552878141403198

In [35]:
np.mean(Winf_time)*60


0.010611130510057723

In [ ]:
np.mean
